In [54]:
from lxml import etree
import pandas as pd
from datetime import date
import requests

In [55]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [56]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [57]:
raiz=doc.getroot()
raiz.tag

'documento'

In [58]:
len(raiz)

4

In [59]:
libro=raiz[0]
libro.tag

'metadatos'

In [60]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [61]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [62]:
libro=raiz[1]
libro[0].text

'\n            '

In [63]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [64]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [65]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [66]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [67]:
type(materias)

list

In [68]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [69]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20201126
dia = 26
mes = 11
año = 2020


In [70]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [71]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20201126'

In [72]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [73]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
        
save_html(r.content, './BOEs/resumen.xml')

In [74]:
#r.encoding

In [75]:
resumen = etree.parse('./BOEs/resumen.xml')

In [76]:
raiz=resumen.getroot()
raiz.tag

'sumario'

In [77]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [78]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [79]:
tabla_resumen = pd.DataFrame()

In [80]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [81]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 4911-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14956],[I. Disposiciones generales]
1,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5178-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14957],[I. Disposiciones generales]
2,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5342-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14958],[I. Disposiciones generales]
3,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4433-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14959],[I. Disposiciones generales]
4,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Medicamentos],"[Real Decreto 957/2020, de 3 de noviembre, por...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14960],[I. Disposiciones generales]
5,[MINISTERIO DE CULTURA Y DEPORTE],[Subvenciones],"[Real Decreto 1042/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14961],[I. Disposiciones generales]
6,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Caja General de Depósitos. Reglamento],"[Real Decreto 937/2020, de 27 de octubre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14962],[I. Disposiciones generales]
7,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14963],[I. Disposiciones generales]
8,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14964],[I. Disposiciones generales]
9,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Seguridad Social],"[Orden ISM/1099/2020, de 23 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14965],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [82]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    f = './BOEs/' + item_URL[-16:] + '.xml'
    save_html(r.content, f)

In [83]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2020-14463.xml', './BOEs/BOE-A-2020-14462.xml', './BOEs/BOE-A-2020-14460.xml', './BOEs/BOE-A-2020-14879.xml', './BOEs/BOE-A-2020-14878.xml', './BOEs/BOE-A-2020-14461.xml', './BOEs/BOE-A-2020-14957.xml', './BOEs/BOE-A-2020-14956.xml', './BOEs/BOE-A-2020-14962.xml', './BOEs/BOE-A-2020-14963.xml', './BOEs/BOE-A-2020-14961.xml', './BOEs/BOE-A-2020-14960.xml', './BOEs/BOE-A-2020-14964.xml', './BOEs/BOE-A-2020-14958.xml', './BOEs/BOE-A-2020-14959.xml', './BOEs/BOE-A-2020-14965.xml', './BOEs/BOE-A-2020-14875.xml', './BOEs/BOE-A-2020-14876.xml', './BOEs/BOE-A-2020-14877.xml', './BOEs/BOE-A-2020-14457.xml']


In [84]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [85]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas},
                                            ignore_index=True)

    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [86]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias
26,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest..."
2,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ..."
1,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ..."
9,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg..."
12,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...
8,"[Comercio, Educación y enseñanza, Industria, S...",[BOE-A-2020-14462],[Catálogo Nacional de Cualificaciones Profesio...
7,"[Agricultura, Educación y enseñanza, Ganadería...",[BOE-A-2020-14463],"[Agricultura, Catálogo Nacional de Cualificaci..."
3,"[Educación y enseñanza, Medio ambiente, Seguri...",[BOE-A-2020-14464],"[Bomberos, Catálogo Nacional de Cualificacione..."
0,"[Educación y enseñanza, Pesca, Trabajo y emple...",[BOE-A-2020-14465],"[Actividades subacuáticas, Acuicultura, Catálo..."
4,"[Comercio, Sanidad]",[BOE-A-2020-14466],"[Comercialización, Epidemias, Material sanitar..."


In [87]:
print(raiz.tag)

documento


In [88]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [89]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Acuerdos internacionales
Créditos Presupuestarios
Ministerio de Sanidad
Organización Mundial de la Salud
Trasplantes de órganos


In [90]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Relaciones internacionales
Sanidad
Sistema financiero


In [91]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [92]:
BOE[7:23]

'BOE-A-2020-14457'

In [93]:
tabla_resumen['Item_id']

0    [BOE-A-2020-14956]
1    [BOE-A-2020-14957]
2    [BOE-A-2020-14958]
3    [BOE-A-2020-14959]
4    [BOE-A-2020-14960]
5    [BOE-A-2020-14961]
6    [BOE-A-2020-14962]
7    [BOE-A-2020-14963]
8    [BOE-A-2020-14964]
9    [BOE-A-2020-14965]
Name: Item_id, dtype: object

In [94]:
tabla_materias['Item_id']

0     [BOE-A-2020-14465]
1     [BOE-A-2020-14459]
2     [BOE-A-2020-14458]
3     [BOE-A-2020-14464]
4     [BOE-A-2020-14466]
5     [BOE-A-2020-14880]
6     [BOE-A-2020-14467]
7     [BOE-A-2020-14463]
8     [BOE-A-2020-14462]
9     [BOE-A-2020-14460]
10    [BOE-A-2020-14879]
11    [BOE-A-2020-14878]
12    [BOE-A-2020-14461]
13    [BOE-A-2020-14957]
14    [BOE-A-2020-14956]
15    [BOE-A-2020-14962]
16    [BOE-A-2020-14963]
17    [BOE-A-2020-14961]
18    [BOE-A-2020-14960]
19    [BOE-A-2020-14964]
20    [BOE-A-2020-14958]
21    [BOE-A-2020-14959]
22    [BOE-A-2020-14965]
23    [BOE-A-2020-14875]
24    [BOE-A-2020-14876]
25    [BOE-A-2020-14877]
26    [BOE-A-2020-14457]
Name: Item_id, dtype: object

In [95]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [96]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [97]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [98]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [99]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 4911-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14956],[I. Disposiciones generales]
1,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5178-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14957],[I. Disposiciones generales]
2,[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5342-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14958],[I. Disposiciones generales]
3,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4433-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14959],[I. Disposiciones generales]
4,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Medicamentos],"[Real Decreto 957/2020, de 3 de noviembre, por...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14960],[I. Disposiciones generales]
5,[MINISTERIO DE CULTURA Y DEPORTE],[Subvenciones],"[Real Decreto 1042/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14961],[I. Disposiciones generales]
6,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Caja General de Depósitos. Reglamento],"[Real Decreto 937/2020, de 27 de octubre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14962],[I. Disposiciones generales]
7,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14963],[I. Disposiciones generales]
8,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14964],[I. Disposiciones generales]
9,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Seguridad Social],"[Orden ISM/1099/2020, de 23 de noviembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-14965],[I. Disposiciones generales]


In [100]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [101]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [102]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias
Item_id,,,,,,,
['BOE-A-2020-14956'],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 4911-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Constitucional, Organización de la Ad...","[Asturias, Organización de las Comunidades Aut..."
['BOE-A-2020-14957'],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5178-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Agricultura, Deporte, Derecho Constitucional,...","[Agricultura, Comités consultivos, Contaminaci..."
['BOE-A-2020-14958'],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],[Recurso de inconstitucionalidad n.º 5342-2020...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Constitucional, Organización de la Ad...","[Comunidades Autónomas, Delegación de atribuci..."
['BOE-A-2020-14959'],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4433-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Constitucional, Sistema tributario, V...","[Ayuntamientos, Catastros, Cuestiones de incon..."
['BOE-A-2020-14960'],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Medicamentos],"[Real Decreto 957/2020, de 3 de noviembre, por...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Organización de la Ad...",[Agencia Española de Medicamentos y Productos ...
['BOE-A-2020-14961'],[MINISTERIO DE CULTURA Y DEPORTE],[Subvenciones],"[Real Decreto 1042/2020, de 24 de noviembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Cultura y ocio],"[Ayudas, Cultura, Epidemias, Instituto Naciona..."
['BOE-A-2020-14962'],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Caja General de Depósitos. Reglamento],"[Real Decreto 937/2020, de 27 de octubre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Derecho Mercantil, Si...","[Aval, Caja General de Depósitos, Depósitos, D..."
['BOE-A-2020-14963'],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Mercantil, Sanidad, Sistema financiero]","[Aval, Ayudas, Empresas, Epidemias, Instituto ..."
['BOE-A-2020-14964'],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],"[Resolución de 25 de noviembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Mercantil, Sanidad, Sistema financiero]","[Aval, Ayudas, Empresas, Epidemias, Instituto ..."


In [103]:
tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [104]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias',], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Alerta
1,['BOE-A-2020-14957'],[Recurso de inconstitucionalidad n.º 5178-2020...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Comités consultivos, Contaminaci...",Agricultura
5,['BOE-A-2020-14961'],"[Real Decreto 1042/2020, de 24 de noviembre, p...",[I. Disposiciones generales],[MINISTERIO DE CULTURA Y DEPORTE],[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Ayudas, Cultura, Epidemias, Instituto Naciona...",Cultura y ocio
11,['BOE-A-2020-14957'],[Recurso de inconstitucionalidad n.º 5178-2020...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Comités consultivos, Contaminaci...",Deporte
4,['BOE-A-2020-14960'],"[Real Decreto 957/2020, de 3 de noviembre, por...",[I. Disposiciones generales],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Medicamentos],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[Agencia Española de Medicamentos y Productos ...,Derecho Administrativo
6,['BOE-A-2020-14962'],"[Real Decreto 937/2020, de 27 de octubre, por ...",[I. Disposiciones generales],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Caja General de Depósitos. Reglamento],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aval, Caja General de Depósitos, Depósitos, D...",Derecho Administrativo
0,['BOE-A-2020-14956'],[Recurso de inconstitucionalidad n.º 4911-2020...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Asturias, Organización de las Comunidades Aut...",Derecho Constitucional
2,['BOE-A-2020-14958'],[Recurso de inconstitucionalidad n.º 5342-2020...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Comunidades Autónomas, Delegación de atribuci...",Derecho Constitucional
3,['BOE-A-2020-14959'],[Cuestión de inconstitucionalidad n.º 4433-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Ayuntamientos, Catastros, Cuestiones de incon...",Derecho Constitucional
21,['BOE-A-2020-14957'],[Recurso de inconstitucionalidad n.º 5178-2020...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Recursos de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Comités consultivos, Contaminaci...",Derecho Constitucional
7,['BOE-A-2020-14963'],"[Resolución de 25 de noviembre de 2020, de la ...",[I. Disposiciones generales],[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Avales],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aval, Ayudas, Empresas, Epidemias, Instituto ...",Derecho Mercantil


In [105]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2020-14956']         Recursos de inconstitucionalidad
['BOE-A-2020-14957']         Recursos de inconstitucionalidad
['BOE-A-2020-14958']         Recursos de inconstitucionalidad
['BOE-A-2020-14959']       Cuestiones de inconstitucionalidad
['BOE-A-2020-14960']                             Medicamentos
['BOE-A-2020-14961']                             Subvenciones
['BOE-A-2020-14962']    Caja General de Depósitos. Reglamento
['BOE-A-2020-14963']                                   Avales
['BOE-A-2020-14964']                                   Avales
['BOE-A-2020-14965']                         Seguridad Social
Name: Epigrafe, dtype: object

In [106]:
tabla_resultados.reset_index(inplace=True)
tabla_resultados['Item_id'].astype(str).str.join('-')

0    [-'-B-O-E---A---2-0-2-0---1-4-9-5-6-'-]
1    [-'-B-O-E---A---2-0-2-0---1-4-9-5-7-'-]
2    [-'-B-O-E---A---2-0-2-0---1-4-9-5-8-'-]
3    [-'-B-O-E---A---2-0-2-0---1-4-9-5-9-'-]
4    [-'-B-O-E---A---2-0-2-0---1-4-9-6-0-'-]
5    [-'-B-O-E---A---2-0-2-0---1-4-9-6-1-'-]
6    [-'-B-O-E---A---2-0-2-0---1-4-9-6-2-'-]
7    [-'-B-O-E---A---2-0-2-0---1-4-9-6-3-'-]
8    [-'-B-O-E---A---2-0-2-0---1-4-9-6-4-'-]
9    [-'-B-O-E---A---2-0-2-0---1-4-9-6-5-'-]
Name: Item_id, dtype: object